**Imports**

In [1]:
import numpy as np

**Helper Functions**

In [2]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates a preconditioned matrix C based on 
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C^(-1)Y
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1) + np.dot(Y, V.T.conj())
    # calculate and return C^(-1)Y
    return np.dot(np.linalg.inv(C), y)

**RQI with APP**

In [3]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
# 
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # keep track of iterations
    i = 0
    # conditional stop variable
    stop = False
    # repeat until stop condition is met 
    # or until the maximum number of iterations is reached
    while(not stop):
        # increase iteration count
        i += 1
        # check that max iterations hasn't been passed
        # if it has been passed, return False to indicate failure
        # also return the max number of iterations
        if i > it:
            return False, False, it
        # compute f()
        x = f(mu, M, y, v)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (v^T)Mv
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, and i
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i

**Test Algorithm**

In [4]:
# create a random matrix M
M = np.random.rand(4, 4)
# calculate the eigenvalues of M
u, v = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract 
# a small value to make it an approximation
mu = u[0]-0.25
# perform the RQIAPP algorithm on M and mu
RQIAPP(M, mu, 1e-8, 100)

((2.0862906647223416+0j),
 array([0.46026298+0.j, 0.22826637+0.j, 0.44428589+0.j, 0.73393631+0.j]),
 4)

In [5]:
# output the real eigenvalue and eigenvector
print(u[0], v[:,0])

(2.086290664722135+0j) [-0.46026298+0.j -0.22826637+0.j -0.44428589+0.j -0.73393631+0.j]
